In [ ]:
#import modul & print atribut of the dataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sn

def load_data():
    dataset = load_digits()
    feature = dataset['data']
    target = dataset['target']

    return feature, target

def preprocess_data(feature, target):
    feature = feature.reshape((len(feature), -1))
    feature = MinMaxScaler().fit_transform(feature)

    target = target.reshape(target.shape[0], -1)
    target = OneHotEncoder(sparse = False).fit_transform(target)
    return feature, target

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\ryang\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\ryang\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\ryang\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\ryang\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\ryang\appdata\local\programs\python\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load f

In [ ]:
#initial layers, weight, & bias
layers = {
    'input': 64,
    'hidden': 15,
    'output': 10
}

weight = {
    'hidden': tf.Variable(tf.random.normal([layers['input'], layers['hidden']])),
    'output': tf.Variable(tf.random.normal([layers['hidden'], layers['output']]))
}

bias = {
    'hidden': tf.Variable(tf.random.normal([layers['hidden']])),
    'output': tf.Variable(tf.random.normal([layers['output']]))
}

In [ ]:
def activation(x):
    return tf.nn.sigmoid(x)

def foward_pass(feature):
    x1 = tf.matmul(feature, weight['hidden']) + bias['hidden']
    y1 = activation(x1)

    x2 = tf.matmul(y1, weight['output']) + bias['output']
    y2 = activation(x2)

    return y2

In [ ]:
feature_temp = tf.placeholder(tf.float32, [None, layers['input']])
target_temp = tf.placeholder(tf.float32, [None, layers['output']])

#learning_rate & epoch
Learning_rate = 0.5
epoch = 30000

output = foward_pass(feature_temp)
error = tf.reduce_mean(0.5 * (target_temp - output)**2)

#optimizer
train = tf.train.GradientDescentOptimizer(Learning_rate).minimize(error)

feature, target = load_data()
feature,target = preprocess_data(feature,target)

#split the data
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size = 0.3)

In [ ]:
arr1 = []
arr2 = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(epoch + 1):
        train_data = {
            feature_temp: feature_train,
            target_temp: target_train
        }
        sess.run(train, feed_dict=train_data)

    if i % 1000 == 0:
        Accuracy = tf.equal(tf.argmax(target_temp, axis=1), tf.argmax(output, axis=1))
        prediction = tf.argmax(output, axis = 1)
        result = tf.reduce_mean(tf.cast(Accuracy, tf.float32))
      
        test_data = {
            feature_temp: feature_test,
            target_temp: target_test
          }
        prediction = sess.run(prediction, feed_dict = test_data)
      
        acc1 = sess.run(result, feed_dict=train_data)
        acc2 = sess.run(result, feed_dict=test_data)
        arr1.append(acc1)
        arr2.append(acc2)
        print(f'Epoch: {i}, Accuracy: {acc2 * 100}%')

    print(f'Final Accuracy: {acc2 * 100}%')

In [ ]:
#Validation Curves

plt.plot(arr1, 'red')
plt.plot(arr2, 'blue')
plt.title('Validation Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
target_test = tf.argmax(target_test, axis=1)

Confusion_Matrix = metrics.confusion_matrix(target, prediction)
df_cm = pd.DataFrame(Confusion_Matrix, range(10), range(10))
plt.figure(figsize=(15,10))
sn.heatmap(df_cm, annot=True)
plt.show()